In [193]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [219]:
train = pd.read_csv('./_data/train.csv', index_col=0)#id를 인덱스로
test = pd.read_csv('./_data/test.csv', index_col=0)

In [195]:
from pycaret.classification import * 

* null이 포함되어있는 feature를 nan으로

In [171]:
# thal = 0 = Null 
train[train['thal']==0]

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
id,,,,,,,,,,,,,,
130,53,0,2,128,216,0,0,115,0,0.0,2,0,0,1


In [172]:
test[test['thal']==0]

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
id,,,,,,,,,,,,,
47,52,1,0,128,204,1,1,156,1,1.0,1,0,0


In [157]:
train.loc[train['thal']==0,'thal'] = np.nan
test.loc[test['thal']==0,'thal'] = np.nan
train.loc[train.thal==1, 'thal'] =np.nan
test.loc[test.thal==1, 'thal'] =np.nan

In [173]:
# ca = 4 = Null
train[train['ca']==4]

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
id,,,,,,,,,,,,,,


In [174]:
test[test['ca']==4]

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
id,,,,,,,,,,,,,
41,52,1,2,138,223,0,1,169,0,0.0,2,4,2
46,43,1,0,132,247,1,0,143,1,0.1,1,4,3
80,58,1,1,125,220,0,1,144,0,0.4,1,4,3
81,38,1,2,138,175,0,1,173,0,0.0,2,4,2
96,38,1,2,138,175,0,1,173,0,0.0,2,4,2


In [220]:
ca_y = test[test['ca']==4]

In [221]:
train_ca = train

y_train_ca = train_ca.pop('ca')
train_ca.pop('target')
x_train_ca = train_ca
x_train_ca

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,thal
id,,,,,,,,,,,,
1,53,1,2,130,197,1,0,152,0,1.2,0,2
2,52,1,3,152,298,1,1,178,0,1.2,1,3
3,54,1,1,192,283,0,0,195,0,0.0,2,3
4,45,0,0,138,236,0,0,152,1,0.2,1,2
5,35,1,1,122,192,0,1,174,0,0.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
147,50,1,2,140,233,0,1,163,0,0.6,1,3
148,51,1,2,94,227,0,1,154,1,0.0,2,3
149,69,1,3,160,234,1,0,131,0,0.1,1,2


In [222]:
ca_y

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
id,,,,,,,,,,,,,
41,52,1,2,138,223,0,1,169,0,0.0,2,4,2
46,43,1,0,132,247,1,0,143,1,0.1,1,4,3
80,58,1,1,125,220,0,1,144,0,0.4,1,4,3
81,38,1,2,138,175,0,1,173,0,0.0,2,4,2
96,38,1,2,138,175,0,1,173,0,0.0,2,4,2


In [236]:
y_test_ca = test
y_test_ca = y_test_ca.drop([41,46,80,81,96])

In [237]:
y_train_ca_all = pd.concat([y_train_ca, y_test_ca.pop('ca')])
y_train_ca_all

id
1      0
2      0
3      1
4      0
5      0
      ..
148    2
149    0
150    2
151    1
152    0
Name: ca, Length: 298, dtype: int64

In [238]:
x_train_ca_all = pd.concat([x_train_ca, y_test_ca])
x_train_ca_all

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,thal
id,,,,,,,,,,,,
1,53,1,2,130,197,1,0,152,0,1.2,0,2
2,52,1,3,152,298,1,1,178,0,1.2,1,3
3,54,1,1,192,283,0,0,195,0,0.0,2,3
4,45,0,0,138,236,0,0,152,1,0.2,1,2
5,35,1,1,122,192,0,1,174,0,0.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
148,64,0,0,130,303,0,1,122,0,2.0,1,2
149,43,0,0,132,341,1,0,136,1,3.0,1,3
150,53,1,0,123,282,0,1,95,1,2.0,1,3


In [ ]:
x_train_ca = pd.concat([x_train_ca, x_test_ca])

In [ ]:
thal_y1 =  train[train['thal']==0]
thal_y2 =  test[test['thal']==0]

train.loc[train['thal']==0,'thal'] = np.nan
test.loc[test['thal']==0,'thal'] = np.nan

tr_train_thal = train.dropna()
tr_y_train_thal = tr_train_thal.pop('thal')
tr_x_train_thal = tr_train_thal
tr_x_train_thal
# train.loc[train.thal==1, 'thal'] =np.nan
# test.loc[test.thal==1, 'thal'] =np.nan

In [134]:
categorical_features =['sex', 'cp', 'exang', 'slope', 'ca', 'thal', 'restecg']

In [135]:
for i in categorical_features:
    print(train[i].value_counts())

1    103
0     48
Name: sex, dtype: int64
0    65
2    47
1    25
3    14
Name: cp, dtype: int64
0    102
1     49
Name: exang, dtype: int64
2    70
1    68
0    13
Name: slope, dtype: int64
0    95
1    31
2    15
3    10
Name: ca, dtype: int64
2.0    84
3.0    58
Name: thal, dtype: int64
1    75
0    75
2     1
Name: restecg, dtype: int64


In [136]:
for i in categorical_features:
    print(test[i].value_counts())

1    104
0     48
Name: sex, dtype: int64
0    78
2    40
1    25
3     9
Name: cp, dtype: int64
0    102
1     50
Name: exang, dtype: int64
2    72
1    72
0     8
Name: slope, dtype: int64
0.0    80
1.0    34
2.0    23
3.0    10
Name: ca, dtype: int64
2.0    82
3.0    59
Name: thal, dtype: int64
1    77
0    72
2     3
Name: restecg, dtype: int64


In [137]:
for i in categorical_features:
    print(f'train[{i}] : {train[i].isna().sum()}')
    print(f'test[{i}] : {test[i].isna().sum()}')

train[sex] : 0
test[sex] : 0
train[cp] : 0
test[cp] : 0
train[exang] : 0
test[exang] : 0
train[slope] : 0
test[slope] : 0
train[ca] : 0
test[ca] : 5
train[thal] : 9
test[thal] : 11
train[restecg] : 0
test[restecg] : 0


In [ ]:
train

In [159]:
x_train = train.dropna()

# 모델 만들기

#### 사용할 데이터
* categorical_features : sex, cp, (exang), slope, ca, thal, restecg
* numeric_features : age, (trestbps), (thalach), (fbs), (oldpeak), (chol)

In [197]:
SEED = 32

In [241]:
ignore_features = []
categorical_features = ['sex', 'cp', 'slope','exang', 'restecg', 'thal']
numeric_features = ['age', 'trestbps', 'thalach', 'fbs','oldpeak', 'chol']

In [119]:
# ignore_features = ['fbs','oldpeak', 'chol']
# categorical_features = ['sex', 'cp', 'slope', 'ca', 'thal','exang','restecg']
# numeric_features = ['age','trestbps', 'thalach']

In [239]:
ca_train = pd.concat([x_train_ca_all,y_train_ca_all], axis=1)
ca_train

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,thal,ca
id,,,,,,,,,,,,,
1,53,1,2,130,197,1,0,152,0,1.2,0,2,0
2,52,1,3,152,298,1,1,178,0,1.2,1,3,0
3,54,1,1,192,283,0,0,195,0,0.0,2,3,1
4,45,0,0,138,236,0,0,152,1,0.2,1,2,0
5,35,1,1,122,192,0,1,174,0,0.0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,64,0,0,130,303,0,1,122,0,2.0,1,2,2
149,43,0,0,132,341,1,0,136,1,3.0,1,3,0
150,53,1,0,123,282,0,1,95,1,2.0,1,3,2


In [244]:
clf = setup(data=ca_train, 
            target='ca', 
            ignore_features=ignore_features,           
            categorical_features=categorical_features, 
            numeric_features=numeric_features,
            remove_outliers=True,
            #outliers_threshold=0.1,
            normalize=True,        
            normalize_method='zscore',    #minmax, zscore, maxabs, robust
            feature_selection=True,
            imputation_type='iterative',
          #  categorical_imputation='mode',
            iterative_imputation_iters=10,            
            categorical_iterative_imputer='lightgbm',
            trigonometry_features=True,
            session_id=SEED, 
            silent=True
            #profile=True
            ) 

,Description,Value
0,session_id,32
1,Target,ca
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(298, 13)"
5,Missing Values,False
6,Numeric Features,6
7,Categorical Features,6
8,Ordinal Features,False
9,High Cardinality Features,False


In [245]:
best_models = compare_models(sort='Accuracy', n_select=3, fold=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.6105,0.0000,0.3222,0.5448,0.5489,0.1794,0.2072,0.0080
rf,Random Forest Classifier,0.6105,0.6505,0.2863,0.4666,0.5167,0.1267,0.1573,0.2120
qda,Quadratic Discriminant Analysis,0.6053,0.5000,0.2500,0.3663,0.4564,0.0000,0.0000,0.0120
dummy,Dummy Classifier,0.6053,0.5000,0.2500,0.3663,0.4564,0.0000,0.0000,0.0080
lr,Logistic Regression,0.6000,0.6897,0.3368,0.5504,0.5502,0.1902,0.2096,0.0140
knn,K Neighbors Classifier,0.6000,0.5843,0.2869,0.4912,0.5261,0.1419,0.1630,0.0600
lda,Linear Discriminant Analysis,0.5842,0.6851,0.3406,0.5474,0.5449,0.1838,0.1959,0.0080
et,Extra Trees Classifier,0.5842,0.6305,0.3058,0.4875,0.5274,0.1485,0.1545,0.2160
svm,SVM - Linear Kernel,0.5632,0.0000,0.3684,0.5109,0.5029,0.1836,0.2221,0.0360
gbc,Gradient Boosting Classifier,0.5526,0.6266,0.2590,0.4442,0.4894,0.0866,0.0973,0.0980


In [214]:
tuned_top3 = [tune_model(i) for i in best_models]
blender = blend_models(tuned_top3)
stacker = stack_models(tuned_top3)
best_auc_model = automl(optimize = 'Accuracy')
finalized_model = finalize_model(best_auc_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7273,0.0000,0.5000,0.6273,0.6453,0.3265,0.4472
1,0.7273,0.0000,0.5000,0.6273,0.6453,0.3265,0.4472
2,0.6364,0.8207,0.3393,0.6591,0.6303,0.2787,0.2902
3,0.7273,0.6409,0.5000,0.5404,0.6174,0.4107,0.4719
4,0.5455,0.8152,0.5179,0.6000,0.5455,0.2763,0.2965
5,0.7000,0.0000,0.7381,0.7333,0.7051,0.4231,0.4335
6,0.5000,0.0000,0.2381,0.4375,0.4667,-0.1905,-0.2085
7,0.6000,0.0000,0.2143,0.5250,0.5600,0.0698,0.0759
8,0.7000,0.0000,0.5833,0.8500,0.7524,0.3750,0.4125
9,0.8000,0.0000,0.6667,0.6444,0.7125,0.4444,0.5560


In [246]:
predictions = predict_model(data=ca_y, estimator=finalized_model)
predictions

ValueError: Number of features of the model must match the input. Model n_features is 21 and input n_features is 15 

In [ ]:
submission['target'] = predictions['Label']
submission

In [ ]:
submission.to_csv('./submission9.csv')

In [ ]:
clf = setup(data=train, 
            target='target', 
            ignore_features=ignore_features,           
            categorical_features=categorical_features, 
            numeric_features=numeric_features,
            remove_outliers=True,
            #outliers_threshold=0.1,
            normalize=True,        
            normalize_method='zscore',    #minmax, zscore, maxabs, robust
            feature_selection=True,
            #feature_selection_threshold=0.9,
            imputation_type='iterative',
            categorical_imputation='mode',
            iterative_imputation_iters=10,            
            categorical_iterative_imputer='xgboost',
            trigonometry_features=True,
            session_id=SEED, 
            silent=True
            #profile=True
            ) 